<a href="https://colab.research.google.com/github/praffuln/langchain/blob/master/langchain_20_ControlTone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google AI chat models


In [ ]:
!pip install langchain
!pip install openAI
!pip install wikipedia
!pip install huggingface_hub
!pip install InstructorEmbedding
!pip install google-search-results
!pip install unstructured
!pip install libmagic
!pip install python-magic
!pip install python-magic-bin
#Install faiss Packages
!pip install faiss-cpu
!pip install sentence-transformers
!pip install wolframalpha
!pip install pypdf
!pip install youtube-transcript-api
!pip install pytube
!pip install python-dotenv
!pip install pinecone-client
!pip install kor
!pip install markdownify


# Google AI chat models


In [11]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI




# API KEY

In [8]:
os.environ["GOOGLE_API_KEY"] = ''

# Replace these values with your own Twitter API credentials
TWITTER_API_KEY = ''
TWITTER_API_KEY_SECRET = ''
TWITTER_ACCESS_TOKEN = ''
TWITTER_ACCESS_TOKEN_SECRET = ''



In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result)
print(result.content)



# Instructing LLMs To Match Tone

LLMs that generate text are awesome, but what if you want to edit the tone/style it responds with?

We've all seen the pirate examples, but it would be awesome if we could tune the prompt to match the tone of specific people?

Below is a series of techniques aimed to generate text in the tone and style you want. No single technique will likely be exactly what you need, but I guarantee you can iterate with these tips to get a solid outcome for your project.

But Greg, what about fine tuning? Fine tuning would likely give you a fabulous result, but the barriers to entry are too high for the average developer (as of May '23). I would rather get the 87% solution today rather than not ship something. If you're doing this in production and your differentiator is your ability to adapt to different styles you'll likely want to explore fine tuning.

If you want to see a demo video of this, check out the Twitter post. For a full explination head over to YouTube.

4 Levels Of Tone Matching Techniques:

**Simple**: As a human, try and describe the tone you would like

**Intermediate**: Include your description + examples

**AI-Assisted**: Ask the LLM to extract tone, use their output in your next prompt

**Technique Fusion**: Combine multiple techniques to mimic tone

**Today's Goal**: Generate tweets mimicking the style of online personalities. You could customize this code to generate emails, chat messages, writing, etc.

First let's import our packages



In [12]:
# LangChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate

# Environment Variables
import os
from dotenv import load_dotenv

# Twitter
import tweepy

load_dotenv()


False

# Method #1: Simple - Describe the tone you would like
Our first method is going to be simply describing the tone we would like.

Let's try a few exmaples



In [13]:
prompt = """
Please create me a tweet about going to the park and eating a sandwich.
"""

output = llm.predict(prompt)
print (output)


Enjoying a beautiful day at the park with a delicious sandwich in hand. The sun is shining, the birds are singing, and the sandwich is to die for. #ParkLife #SandwichLove #PerfectDay


Not bad, but I don't love the emojis and I want it to use more conversational modern language.

Let's try again



In [14]:
prompt = """
Please create me a tweet about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Use simple language a 5 year old would understand
"""

output = llm.predict(prompt)
print (output)


"Mommy and I went to the park today! We brought a yummy sandwich and ate it on a big, cozy blanket. I made a sand castle and played with the ducks. It was the bestest day!"


Ok cool! The tone has changed. Not bad but now I want it to sound like a specific person. Let's try Bill Gates:



In [15]:
prompt = """
Please create me a tweet about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Bill Gates
"""

output = llm.predict(prompt)
print (output)


Today, I ventured to the park and partook in a delightful picnic. As I sunk my teeth into a savory sandwich, the crunch of the bread and the burst of flavors filled my senses. It was a moment of pure bliss, surrounded by nature's beauty. Life's simple pleasures are often the most fulfilling.


It's ok, I'd give the response a C+ right now.

Let's give some example tweets so the model can better match tone/style.

⭐ Important Tip: When you're giving examples, make sure to have the examples the same as the desired output format. Ex: Tweets > Tweets, Email > Email. Don't do Tweets > Email



# Method #2: Intermediate - Specify your tone description + examples

Examples speak a thousand words. Let's pass a few along with our instructions to see how it goes



# Get a users Tweets
Next let's grab a users tweets. We'll do this in a function so it's easy to pull them later

Since we are live Tweets, you'll need to grather some Twitter api keys. You can get these on the Twitter Developer Portal. The free tier is fine, but watch out for rate limits.



In [29]:
# We'll query 70 tweets because we end up filtering out a bunch, but we'll only return the top 12.
# We will also only use a subset of the top tweets later
def get_original_tweets(screen_name, tweets_to_pull=20, tweets_to_return=5):

    # Tweepy set up
    auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_KEY_SECRET)
    auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth)

    tweets = []

    tweepy_results = api.user_timeline(screen_name="billgates", count=20)

    public_tweets = api.home_timeline()
    print(public_tweets)

    # tweepy_results = tweepy.Cursor(api.user_timeline,
    #                                screen_name=screen_name,
    #                                tweet_mode='extended',
    #                                exclude_replies=True).items(tweets_to_pull)

    # Run through tweets and remove retweets and quote tweets so we can only look at a user's raw emotions
    # for status in tweepy_results:
    #     if not hasattr(status, 'retweeted_status') and not hasattr(status, 'quoted_status'):
    #         tweets.append({'full_text': status.full_text, 'likes': status.favorite_count})


    # # Sort the tweets by number of likes. This will help us short_list the top ones later
    # sorted_tweets = sorted(tweets, key=lambda x: x['likes'], reverse=True)

    # # Get the text and drop the like count from the dictionary
    # full_text = [x['full_text'] for x in sorted_tweets][:tweets_to_return]

    # # Conver the list of tweets into a string of tweets we can use in the prompt later
    # example_tweets = "\n\n".join(full_text)

    return public_tweets


In [ ]:
user_screen_name = 'billgates'  # Replace this with the desired user's screen name
users_tweets = get_original_tweets(user_screen_name)
print(users_tweets)


In [31]:
## don't have write access.
users_tweets ='''
These numbers prove why India plays such a crucial role in the world’s fight to improve health, reduce poverty, prevent climate change, and more. https://t.co/xMpmcoYQhi

Mann ki Baat has catalyzed community led action on sanitation, health, women’s economic empowerment and other issues linked to the Sustainable Development Goals. Congratulations @narendramodi on the 100th episode. https://t.co/yg1Di2srjE

The development of AI is as fundamental as the creation of the microprocessor, the personal computer, the Internet, and the mobile phone. It will change the way people work, learn, travel, get health care, and communicate with each other. https://t.co/uuaOQyxBTl

I just returned from my visit to India, and I can’t wait to go back again. I love visiting India because every trip is an incredible opportunity to learn. Here are some photos from my trip and some of the stories behind them: https://t.co/We6PtJWDnp https://t.co/QxZW7gfUmI

Superintelligent AIs are in our future. Compared to a computer, our brains operate at a snail’s pace. An electrical signal in the brain moves at ___________ the speed of the signal in a silicon chip. Check your answer here: https://t.co/wqZG1BdoTc

Thinking of President Carter and his family. This is a lovely tribute to one of his biggest accomplishments. https://t.co/g53c4ty0qI

Uganda’s maternal mortality rate is at least double the global average. That's why Eva Nangalo has dedicated her life to making childbirth in the country safer for everyone involved. https://t.co/29AjdJehNY

I am so impressed with Eva Nangalo—it’s hard not to be. She’s spent decades making childbirth safer in Uganda for everyone involved, and she’s become a mentor to countless other midwives in the process. https://t.co/79RHbrCt01

I recently had the chance to test drive—or test ride, I guess—one of @wayve_ai’s autonomous vehicles. It was a pretty wild ride: https://t.co/PrwrxU49dd https://t.co/NtnkVx7sBx

When I transitioned from @Microsoft to working full-time at the @GatesFoundation, I finally had the time to learn more about physics, chemistry, biology, and other sciences. So, I looked around for the best books and read as many of them as I could find. https://t.co/z2D5xGSeMj

As big as the problems facing the world are right now, my visit to India reminded me that our capacity to solve them is even bigger: https://t.co/zp7XfRIpV9 https://t.co/aFHUu987u3

I’m grateful for the Lauder family’s dedication to solving Alzheimer’s. https://t.co/vX0qtjBFxt

'''

# Pass the tweets as examples


In [ ]:
template = """
Please create me a tweet about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Bill Gates

% START OF EXAMPLE TWEETS TO MIMIC
{example_tweets}
% END OF EXAMPLE TWEETS TO MIMIC

YOUR TWEET:
"""

prompt = PromptTemplate(
    input_variables=["example_tweets"],
    template=template,
)

final_prompt = prompt.format(example_tweets=users_tweets)

print (final_prompt)


In [33]:
output = llm.predict(final_prompt)
print (output)


Enjoying a delightful afternoon at the park with a scrumptious sandwich, savoring each bite while surrounded by nature's beauty. #PeacefulHarmony #SimplePleasures


Wow! Ok now that is starting to get somewhere. Not bad at all! Sounds like Bill is in the room with us now.

Let's see if we can refine it even more.



# Method #3: AI-Assisted: Ask the LLM help with tone descriptions

Turns out I'm not great at describing tone. Examples are a good way to help, but can we do more? Let's find out.

I want to have the model tell me what tone it sees, then use that output as an input to the final prompt where I ask it to generate a tweet.

Almost like reverse engineering tone.

Why don't I do this all in one step? You likely could, but it would be nice to save this "tone" description for future use. Plus, I don't want the model to take too many logic jumps in a single response.

I first thought, 'well... what are the qualities of tone I should have it describe?'

Then I thought, Greg, c'mon man, you know better than that, see if the LLM has a good sense of what tone qualities there are. Duh.

Let's see what are the qualities of tone we should extract



In [35]:
prompt = """
Can you please generate a list of tone attributes and a description to describe a piece of writing by? please include example also.

Things like pace, mood, etc.

Respond with nothing else besides the list
"""

how_to_describe_tone = llm.predict(prompt)
print (how_to_describe_tone)


- **Tone:** The overall emotional atmosphere of a piece of writing.
- **Mood:** The emotional state that the reader is left with after reading a piece of writing.
- **Pace:** The speed at which a piece of writing moves.
- **Rhythm:** The flow and cadence of a piece of writing.
- **Voice:** The personality of the writer as expressed through their writing.
- **Diction:** The word choice of a writer.
- **Syntax:** The structure of a sentence or phrase.
- **Figurative Language:** The use of words or phrases that create a vivid image or idea in the reader's mind.
- **Imagery:** The use of words or phrases that create a sensory experience for the reader.
- **Symbolism:** The use of objects, characters, or actions to represent something else.
- **Allusion:** The reference to a well-known person, place, thing, or event.
- **Irony:** The use of words or situations to create a contrast between what is expected and what actually happens.


In [36]:
def get_authors_tone_description(how_to_describe_tone, users_tweets):
    template = """
        You are an AI Bot that is very good at generating writing in a similar tone as examples.
        Be opinionated and have an active voice.
        Take a strong stance with your response.

        % HOW TO DESCRIBE TONE
        {how_to_describe_tone}

        % START OF EXAMPLES
        {tweet_examples}
        % END OF EXAMPLES

        List out the tone qualities of the examples above
        """

    prompt = PromptTemplate(
        input_variables=["how_to_describe_tone", "tweet_examples"],
        template=template,
    )

    final_prompt = prompt.format(how_to_describe_tone=how_to_describe_tone, tweet_examples=users_tweets)

    authors_tone_description = llm.predict(final_prompt)

    return authors_tone_description


In [37]:
authors_tone_description = get_authors_tone_description(how_to_describe_tone, users_tweets)
print (authors_tone_description)


- **Informative**: The examples provide information on various topics such as India's role in global development, the impact of Mann ki Baat, the significance of AI, and the work of Eva Nangalo in improving maternal health in Uganda.


- **Engaging**: The examples use personal anecdotes, vivid descriptions, and interactive elements to capture the reader's attention and make the content more relatable.


- **Optimistic**: The examples convey a positive outlook on the world and highlight the potential for progress and solutions to global challenges.


- **Thought-provoking**: The examples encourage readers to think critically about important issues and consider different perspectives.


- **Inspiring**: The examples showcase the work of individuals and organizations that are making a positive impact on the world, inspiring readers to take action and contribute to global progress.


- **Passionate**: The examples reflect the author's strong beliefs and commitment to addressing global issu

Great, now that we have Bill Gate's tone style, let's put those tone instructions in with the prompt we had before to see if it helps



In [38]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below
 - Do not use hashtags or emojis
 - Respond in the tone of Bill Gates

% Description of the authors tone:
{authors_tone_description}

% Authors writing samples
{tweet_examples}

% YOUR TASK
Please create a tweet about going to the park and eating a sandwich.
"""

prompt = PromptTemplate(
    input_variables=["authors_tone_description", "tweet_examples"],
    template=template,
)

final_prompt = prompt.format(authors_tone_description=authors_tone_description, tweet_examples=users_tweets)
llm.predict(final_prompt)


"Just spent a delightful day at the park, basking in the sunshine and breathing in the fresh air. Enjoyed a delightful picnic lunch, savoring every bite of my sandwich while surrounded by nature's beauty. It's these simple moments that truly enrich life. #ParkBliss #NatureLovers"

Hmm, better! Not wonderful.

Let's try out the final approach



# Method 4 - Technique Fusion: Combine multiple techniques to mimic tone
After a lot of experimentation I've found the below tips to be helpful

**Don't reference the word 'tweet' in your prompt** - The model has an extremely strong bias towards what a 'tweet' is an will overload you with hashtags and emojis. Rather call it "a short public statement around 300 characters"

**Ask the LLM for similar sounding authors** - Whereas model bias on the word 'tweet' (point #1) isn't great, we can use it in our favor. Ask the LLM which authors the writing style sounds like, then ask the LLM to respond like that author. It's not great that the model is basing the tone off another person but it's a great 89% solution. I learned of this technique from Scott Mitchell.
Examples should be in the output format you want - Everyone has a different voice. Twitter voice, email voice etc. Make sure that the examples you feed to the prompt are the same voice as the output you want. Ex: Don't exect a book to be written from twitter examples.

**Use the Language Model to extract tone** - If you are at a loss for words on how to describe the tone you want, have the language model describe it for you. I found I needed to tell the model to be opinionated, it was too grey-area before.

**Topics matter** - Have the model propose topics first, then give you a tweet. Not only is it better to have things the author would actually talk about, but it's also really good to keep the model on track by having it outline the topics first then respond

Let's first identify authors the model thinks the example tweets sound like, then we'll reference those later. Keep in mind this isn't a true classification exercise and the point isn't to be 100% correct on similar people, it's to get a reference to who the model thinks is similar so we can use that inuition for instructions later.



In [41]:
def get_similar_public_figures(tweet_examples):
    template = """
    You are an AI Bot that is very good at identifying authors, public figures, or writers whos style matches a piece of text
    Your goal is to identify which authors, public figures, or writers sound most similar to the text below

    % START EXAMPLES
    {tweet_examples}
    % END EXAMPLES

    Which authors (list up to 1 if necessary) most closely resemble the examples above? Only respond with the names separated by commas
    """

    prompt = PromptTemplate(
        input_variables=["tweet_examples"],
        template=template,
    )

    # Using the short list of examples so save on tokens and (hopefully) the top tweets
    final_prompt = prompt.format(tweet_examples=tweet_examples)

    authors = llm.predict(final_prompt)
    return authors


In [42]:
authors = get_similar_public_figures(users_tweets)
print (authors)


Bill Gates


Ok that's not that exciting! Becuase we used Bill Gates' example tweets. Trust me that it's better with less-known people. We'll try this more later.

At last, the final output. Let's bring it all together in a single prompt. Notice the 2 step process in the "your task" section below



In [43]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below

% Mimic These Authors:
{authors}

% Description of the authors tone:
{tone}

% Authors writing samples
{example_text}
% End of authors writing samples

% YOUR TASK
1st - Write out topics that this author may talk about
2nd - Write a concise passage (under 300 characters) as if you were the author described above
"""

method_4_prompt_template = PromptTemplate(
    input_variables=["authors", "tone", "example_text"],
    template=template,
)

# Using the short list of examples so save on tokens and (hopefully) the top tweets
final_prompt = method_4_prompt_template.format(authors=authors,
                                               tone=authors_tone_description,
                                               example_text=users_tweets)


In [44]:
output = llm.predict(final_prompt)
print (output)


**Topics:**

- Global development and poverty reduction
- Healthcare and maternal health
- Education and technology
- Climate change and sustainability
- Artificial Intelligence and its impact on society
- Philanthropy and social responsibility
- Inspirational stories of individuals making a positive impact
- Sustainable Development Goals and the role of India

**Passage:**

"Innovation and collaboration are key to tackling global challenges. From AI shaping industries to community-led initiatives transforming lives, each step forward brings us closer to a brighter future for all."
